In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import numpy as np
import sqlite3
from datetime import date,datetime,timedelta
import schedule

In [2]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

In [ ]:
def LinkExtractor():
    a=0
    while True:    
        # set up connection to sqlite3 database
        conn = sqlite3.connect('posts.db')
        c = conn.cursor()

        # create a table to store links
        c.execute('''CREATE TABLE IF NOT EXISTS posts (link TEXT PRIMARY KEY, is_crawled INTEGER, Created_date TEXT )''')

        # specify the URL of the Facebook page
        link='https://www.sheypoor.com/%D8%A7%D8%B3%D8%AA%D8%A7%D9%86-%D8%AA%D9%87%D8%B1%D8%A7%D9%86/%D8%A7%D9%85%D9%84%D8%A7%DA%A9/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4-%D8%AE%D8%A7%D9%86%D9%87-%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86'

        # make a request to the URL
        html_text=requests.get(link+'?p='+str(a+1), headers=headers)

        # parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_text.content,'lxml')

        # find all the links to posts on the page
        posts=soup.find_all('article',class_='serp-item list')

        # save the links to the database
        for link in posts:
            post_link = link.find('a').get('href')
            post_date = datetime.strptime(link.find('time')['datetime'], '%Y-%m-%d %H:%M:%S.%f').date()
            if post_date < date.today()- timedelta(days=1):
                break
            else:
                c.execute("INSERT OR IGNORE INTO posts (link,is_crawled,Created_date) VALUES (?,?,?)", (post_link,0,post_date))
                conn.commit()
        if post_date < date.today()- timedelta(days=1):
        # We've gone past the target date, so we can exit the loop
            break
        # close the database connection
        conn.close()
        a=a+1
        print(a,end="\r")

        
        
schedule.every().day.at("00:10").do(LinkExtractor)

while True:
    schedule.run_pending()
    time.sleep(60) # Wait for 1 minute